# DEMO de Despligue de Modelos

In [ ]:
!pip install -U transformers accelerate

In [ ]:
import torch, requests, gc
from transformers import pipeline, AutoProcessor, AutoModelForCausalLM
from PIL import Image



###   LLama 3.2 3B Instruct
(Está bloqueado este modelo así que ha sido cambiado por https://huggingface.co/MaziyarPanahi/calme-3.1-instruct-3b)





1.   **Cargar Modelo en memoria**



In [ ]:
model_id = "MaziyarPanahi/calme-3.1-instruct-3b"
pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="cuda:0",
)



2.   **Preparar prompt**



In [ ]:
messages = [
    {"role": "system", "content": "Eres un asistente que responde a todas las preguntas con sinceridad y brevedad."},
    {"role": "user", "content": "Que es un MOOC y para que sirve?"},
]



3.   **Inferencia**



In [ ]:
outputs = pipe(
    messages,
    max_new_tokens=500,
    temperature=1,
)
print(outputs[0]["generated_text"][-1]['content'])

4.   **Limpieza de Memoria**

In [ ]:
del pipe, messages, outputs
gc.collect()
torch.cuda.empty_cache()



###   Florence 2 Base (Multimodal)



1.   **Cargar Modelo en memoria**

In [ ]:
model = AutoModelForCausalLM.from_pretrained("microsoft/Florence-2-base", torch_dtype=torch.float16, trust_remote_code=True).to("cuda:0")
processor = AutoProcessor.from_pretrained("microsoft/Florence-2-base", trust_remote_code=True)

2.   **Preparar prompt**

In [ ]:
url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/0052a70beed5bf71b92610a43a52df6d286cd5f3/diffusers/rabbit.jpg"
image = Image.open(requests.get(url, stream=True).raw)
inputs = processor(text="Describe la imagen", images=image, return_tensors="pt").to("cuda:0", torch.float16)

3.   **Inferencia**


In [ ]:
generated_ids = model.generate(
    input_ids=inputs["input_ids"],
    pixel_values=inputs["pixel_values"],
    max_new_tokens=1024,
    do_sample=False,
    num_beams=3,
)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
parsed_answer = processor.post_process_generation(generated_text, task="Describe la imagen", image_size=(image.width, image.height))
print(parsed_answer)